In [1]:
import re
import qiskit
import json
import time
import os
import numpy as np
from qiskit import transpile, QuantumCircuit
import time

from fast_generator import fc_tree_commute_lookahead_fast
from benchmarks.UCCSD_entanglers import generate_UCCSD_entanglers
from circuit_generator import construct_qcc_circuit

import numpy as np
from rustiq import pauli_network_synthesis, Metric
from rustiq.utils import entangling_count, entangling_depth
from qiskit_ibm_runtime.fake_provider import FakeSherbrooke
import rustiq
from utilities import load_sycamore_coupling_map

In [2]:
def convert_to_circuit(circuit: list) -> QuantumCircuit:
    """
    Converts a circuit into a CNOT + H + S + RZ(theta) to a quantumcircuit
    """
    num_qubits=len(set([num for op in circuit for num in op[1]]))
    new_circuit=QuantumCircuit(num_qubits)
    for gate in circuit:
        op=gate[0]
        params=gate[1]
        assert op in ["CNOT", "CZ", "H", "S", "Sd", "X", "Z", "SqrtX", "SqrtXd"], f"op not allowed {op}"
        if op=="CNOT":
            new_circuit.cx(params[0], params[1])
        elif op=="CZ":
            new_circuit.cz(params[0], params[1])
        elif op=="H":
            new_circuit.h(params[0])
        elif op=="S":
            new_circuit.s(params[0])
        elif op=="Sd":
            new_circuit.sdg(params[0])
        elif op=="X":
            new_circuit.x(params[0])
        elif op=="Z":
            new_circuit.z(params[0])
        elif op=="SqrtX":
            new_circuit.sx(params[0])
        elif op=="SqrtXd":
            new_circuit.sxdg(params[0])
    return new_circuit

In [3]:
#Compare json files in a specific folder
def run_experiment_folder(folder_path = None, filename = None, save_output = False, lookahead_size = 5):

    if filename == None:
        file_list = os.listdir(folder_path)
    else:
        file_list = [filename]
    # Iterate over all files in the folder
    for filename in file_list:
        # Check if the file is a JSON file
        if filename.endswith(".json"):
            results = []
            # Print the filename
            print(filename)
            with open(folder_path + '/' + filename, "r") as file:
                paulis = json.load(file)

            # Function to check if a string contains only 'I' characters
            def is_all_identity(pauli):
                return all(char == 'I' for char in pauli)

            # Filter the list to remove all identity Paulis
            test_paulis = [pauli for pauli in paulis if not is_all_identity(pauli)]
            test_params = [0.01 * i for i in range(len(test_paulis))]

            # Measure time for our method
            start_time = time.time()
            opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=lookahead_size)
            opt_qc_f = transpile(opt_qc_f, optimization_level=0, basis_gates=["cx", "sx", "x", "rz"], coupling_map=load_sycamore_coupling_map())
            end_time = time.time()
            our_time = end_time - start_time

            start_time = time.time()
            opt_qc_f2, append_clifford_f2, sorted_entanglers_f2 = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=lookahead_size)
            opt_qiskit = transpile(opt_qc_f2, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"], coupling_map=load_sycamore_coupling_map())
            end_time = time.time()
            combined_time = end_time - start_time
            
            # Measure time for Qiskit method
            start_time = time.time()
            origin_qc = construct_qcc_circuit(entanglers=test_paulis, params=test_params, barrier=False)
            origin_qiskit = transpile(origin_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"], coupling_map=load_sycamore_coupling_map())
            end_time = time.time()
            qiskit_time = end_time - start_time

            # Measure time for RustiQ method
            start_time = time.time()
            circuit = pauli_network_synthesis(test_paulis, Metric.COUNT, True, fix_clifford=True)
            qiskit_circ_rustiq=convert_to_circuit(circuit)
            # print(qiskit_circ_rustiq)
            # can't optimize with level 3 since rustiq returns an optimized circuit missing some single qubit gates.
            qiskit_circ_rustiq = transpile(qiskit_circ_rustiq, optimization_level=0, basis_gates=["cx", "sx", "x", "rz"], coupling_map=load_sycamore_coupling_map())
            end_time = time.time()
            rustiq_time = end_time - start_time
        
            # Collect results
            result = {
                "num_paulis": len(test_paulis),
                "times": {
                    "our_time": our_time,
                    "combined_time": combined_time,
                    "qiskit_time": qiskit_time,
                    "rustiq_time": rustiq_time
                },
                "gate_counts": {
                    "our_method": opt_qc_f.count_ops().get('cx', 0),
                    "combined_method": opt_qiskit.count_ops().get('cx', 0),
                    "qiskit_method": origin_qiskit.count_ops().get('cx', 0),
                    "rustiq_method": qiskit_circ_rustiq.count_ops().get('cx', 0)
                },
                "circuit_entangling_depth": {
                    "our_method": opt_qc_f.depth(lambda instr: len(instr.qubits) > 1),
                    "combined_method": opt_qiskit.depth(lambda instr: len(instr.qubits) > 1),
                    "qiskit_method": origin_qiskit.depth(lambda instr: len(instr.qubits) > 1),
                    "rustiq_method": qiskit_circ_rustiq.depth(lambda instr: len(instr.qubits) > 1)
                },
                "test_paulis_file": f'benchmarks/results_hardware_topology_google_sycamore/test_' + filename
            }
            print(result)
            results.append(result)
            if save_output == True:
                # Save test_paulis to a separate JSON file
                with open(f'benchmarks/results_hardware_topology_google_sycamore/test_' + filename, 'w') as paulis_file:
                    json.dump([test_paulis, results], paulis_file, indent=4)
    

    

In [4]:
#Compare a given list of paulis
def run_experiment_paulis(test_paulis, test_params = None, save_output = False, lookahead_size = 5, filename = "Paulis"):

    results = []

    def is_all_identity(pauli):
        return all(char == 'I' for char in pauli)
    paulis = test_paulis
    # Filter the list to remove all identity Paulis
    test_paulis = [pauli for pauli in paulis if not is_all_identity(pauli)]
    if test_params is None:
        test_params = [0.01 * i for i in range(len(test_paulis))]

    # Measure time for our method
    start_time = time.time()
    opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=lookahead_size)
    opt_qc_f = transpile(opt_qc_f, optimization_level=0, basis_gates=["cx", "sx", "x", "rz"], coupling_map=load_sycamore_coupling_map())
    end_time = time.time()
    our_time = end_time - start_time

    start_time = time.time()
    opt_qc_f2, append_clifford_f2, sorted_entanglers_f2 = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=lookahead_size)
    opt_qiskit = transpile(opt_qc_f2, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"], coupling_map=load_sycamore_coupling_map())
    end_time = time.time()
    combined_time = end_time - start_time
    
    # Measure time for Qiskit method
    start_time = time.time()
    origin_qc = construct_qcc_circuit(entanglers=test_paulis, params=test_params, barrier=False)
    origin_qiskit = transpile(origin_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"], coupling_map=load_sycamore_coupling_map())
    end_time = time.time()
    qiskit_time = end_time - start_time

    # Measure time for RustiQ method
    start_time = time.time()
    circuit = pauli_network_synthesis(test_paulis, Metric.COUNT, True, fix_clifford=True)
    qiskit_circ_rustiq=convert_to_circuit(circuit)
    # print(qiskit_circ_rustiq)
    # can't optimize with level 3 since rustiq returns an optimized circuit missing some single qubit gates.
    qiskit_circ_rustiq = transpile(qiskit_circ_rustiq, optimization_level=0, basis_gates=["cx", "sx", "x", "rz"], coupling_map=load_sycamore_coupling_map())
    end_time = time.time()
    rustiq_time = end_time - start_time
    
    # Collect results
    result = {
        "num_paulis": len(test_paulis),
        "times": {
            "our_time": our_time,
            "combined_time": combined_time,
            "qiskit_time": qiskit_time,
            "rustiq_time": rustiq_time
        },
        "gate_counts": {
            "our_method": opt_qc_f.count_ops().get('cx', 0),
            "combined_method": opt_qiskit.count_ops().get('cx', 0),
            "qiskit_method": origin_qiskit.count_ops().get('cx', 0),
            "rustiq_method": qiskit_circ_rustiq.count_ops().get('cx', 0)
        },
        "circuit_entangling_depth": {
            "our_method": opt_qc_f.depth(lambda instr: len(instr.qubits) > 1),
            "combined_method": opt_qiskit.depth(lambda instr: len(instr.qubits) > 1),
            "qiskit_method": origin_qiskit.depth(lambda instr: len(instr.qubits) > 1),
            "rustiq_method": qiskit_circ_rustiq.depth(lambda instr: len(instr.qubits) > 1)
        },
        "test_paulis_file": f'benchmarks/results_hardware_topology_google_sycamore/test_' + filename
    }
    print(result)
    results.append(result)
    if save_output == True:
        # Save test_paulis to a separate JSON file
        with open(f'benchmarks/results_hardware_topology_google_sycamore/test_' + filename, 'w') as paulis_file:
            json.dump([test_paulis, results], paulis_file, indent=4)
    return sorted_entanglers_f
    

    

In [5]:
#first compare the UCCSD ansatz
electrons_list = [2, 2, 4, 6, 8, 10,] #12]
orbitals_list = [4, 6, 8, 12, 16, 20,] #24]
#First evaluate the UCCSD ansatz:
for i, (x, y) in enumerate(zip(electrons_list, orbitals_list)):
    test_paulis = generate_UCCSD_entanglers(x, y)
    entanglers = run_experiment_paulis(test_paulis = test_paulis, save_output = True, lookahead_size = 5, filename=f"Paulis{len(test_paulis)}.json")

{'num_paulis': 24, 'times': {'our_time': 0.57246994972229, 'combined_time': 0.2892277240753174, 'qiskit_time': 0.23138022422790527, 'rustiq_time': 0.12367057800292969}, 'gate_counts': {'our_method': 84, 'combined_method': 41, 'qiskit_method': 41, 'rustiq_method': 111}, 'circuit_entangling_depth': {'our_method': 64, 'combined_method': 34, 'qiskit_method': 41, 'rustiq_method': 81}, 'test_paulis_file': 'benchmarks/results_hardware_topology_google_sycamore/test_Paulis24.json'}
{'num_paulis': 80, 'times': {'our_time': 1.511960744857788, 'combined_time': 1.1239948272705078, 'qiskit_time': 0.8297836780548096, 'rustiq_time': 0.7031204700469971}, 'gate_counts': {'our_method': 424, 'combined_method': 165, 'qiskit_method': 145, 'rustiq_method': 530}, 'circuit_entangling_depth': {'our_method': 283, 'combined_method': 134, 'qiskit_method': 142, 'rustiq_method': 386}, 'test_paulis_file': 'benchmarks/results_hardware_topology_google_sycamore/test_Paulis80.json'}
{'num_paulis': 320, 'times': {'our_tim

In [ ]:
#Then compare the Hamiltonian simulation paulis in HS_paulis folder
run_experiment_folder(folder_path = "benchmarks/HS_paulis", save_output = True, lookahead_size = 10)

benzene.json


KeyboardInterrupt: 

In [ ]:
#Then compare the MAXCUT paulis in max_cut_paulis folder

run_experiment_folder(folder_path = "benchmarks/max_cut_paulis", save_output = True, lookahead_size = 10)

max_cut_benchmark_n10_e45_l20.json


KeyboardInterrupt: 

In [ ]:
#Labs

run_experiment_folder(folder_path = "benchmarks/labs_paulis", save_output = True, lookahead_size = 10)

labs_n10_layers1.json
{'num_paulis': 80, 'times': {'our_time': 6.7952940464019775, 'combined_time': 5.9363415241241455, 'qiskit_time': 3.450293779373169, 'rustiq_time': 1.2646331787109375}, 'gate_counts': {'our_method': 797, 'combined_method': 779, 'qiskit_method': 1831, 'rustiq_method': 821}, 'circuit_entangling_depth': {'our_method': 397, 'combined_method': 409, 'qiskit_method': 873, 'rustiq_method': 396}, 'test_paulis_file': 'benchmarks/results_hardware_topology/test_labs_n10_layers1.json'}
labs_n10_layers3.json


KeyboardInterrupt: 